<a href="https://colab.research.google.com/github/cwagner2325/WiseInbox_ML/blob/main/Wise_Inbox_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 699.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import random
import os
from google.colab import userdata
from pyngrok import conf
from time import sleep
import sklearn
from sklearn.pipeline import Pipeline

In [3]:
app = FastAPI()

In [4]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
class model_input(BaseModel):

    Email : str
    Sender: str
    Subject : str

In [6]:
with open('/content/drive/MyDrive/XGBModel4', 'rb') as model_file:
    classifier = pickle.load(model_file)

with open('/content/drive/MyDrive/XGBModel4Vectorizer', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

In [7]:
def predictEmail(input_text):
  transformed_email = vectorizer.transform([input_text])
  prediction = classifier.predict(transformed_email)
  probabilities = classifier.predict_proba(transformed_email)

  PHISHING = 1;
  SAFE = 0

  if prediction == SAFE:
      print("Predicted class: Safe Email")
      print("Probabilities:", probabilities.item(0))
      return "Safe Email", probabilities.item(0)
  elif prediction == PHISHING:
      print("Predicted class: Phishing Email")
      print("Probabilities:", probabilities.item(1))
      return "Phishing Email", probabilities.item(1)

In [8]:
@app.post('/email_prediction')
def email_prediction(input_parameters : model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    emailBody = input_dictionary['Email']
    sender = input_dictionary['Sender']
    subject = input_dictionary['Subject']

    predictionResult, confidence = predictEmail(emailBody)

    if predictionResult == "Phishing Email":
        # If confidence is 1 - prediction is 50
        # If confidence is .7 - prediction is 68
        # If confidence is .5 - prediciton is 80
        prediction = 110 - (60 * confidence)
    else:
        prediction = 80 + (20 * confidence)

    return {"prediction": prediction}

In [ ]:
os.environ["TOKEN"] = userdata.get("TOKEN")
ngrok.set_auth_token(os.environ["TOKEN"])

ngrok_tunnel = ngrok.connect(8000)

# print('Public URL:', ngrok_tunnel.public_url)
# print("Here is a test command to run in a terminal: ")
# print('curl --insecure -X POST -H "Content-Type: application/json" -d \'{\n' +
#       '\t"Email": "example@example.com",\n\t"Sender": "John Doe",\n\t"Subject": "Important News" \n}\' ' +
#       ngrok_tunnel.public_url + "/email_prediction")

print("Running at public URL:", ngrok_tunnel.public_url + "/email_prediction")

nest_asyncio.apply()
uvicorn.run(app, port=8000, log_level="warning")

Running at public URL: https://0f35-34-16-153-70.ngrok-free.app/email_prediction
Predicted class: Safe Email
Probabilities: 0.8499016761779785
Predicted class: Safe Email
Probabilities: 0.9925278425216675
Predicted class: Safe Email
Probabilities: 0.9594492316246033
Predicted class: Safe Email
Probabilities: 0.9653391242027283
Predicted class: Safe Email
Probabilities: 0.9839643836021423
Predicted class: Safe Email
Probabilities: 0.9004444479942322
Predicted class: Safe Email
Probabilities: 0.9989251494407654
Predicted class: Safe Email
Probabilities: 0.6039575338363647
Predicted class: Phishing Email
Probabilities: 0.8161872625350952
Predicted class: Phishing Email
Probabilities: 0.6144920587539673
Predicted class: Phishing Email
Probabilities: 0.8620231747627258
Predicted class: Phishing Email
Probabilities: 0.9032108783721924
Predicted class: Phishing Email
Probabilities: 0.8186761140823364
Predicted class: Safe Email
Probabilities: 0.783378541469574
Predicted class: Phishing Email
